# Imports

In [1]:
import datetime
import json
import os
import pickle
from itertools import chain, islice
from multiprocessing import Pool, cpu_count
from pathlib import Path
from pprint import pprint

with open("formation_energy.json", "r") as f:
    formation_energy_pd = json.load(f)
import dateutil.parser
import findspark
import lmdb
import numpy as np
import pyspark
from ase.atoms import Atoms

from functools import partial
from ase.io.cfg import read_cfg
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    BooleanType,
    DoubleType,
    FloatType,
    IntegerType,
    LongType,
    StringType,
    StructField,
    StructType,
    TimestampType,
)

from colabfit.tools.configuration import AtomicConfiguration

from colabfit.tools.database import DataManager, PGDataLoader
from colabfit.tools.property import Property, property_object_schema
from colabfit.tools.configuration import config_schema
from colabfit.tools.property_definitions import (
    atomic_forces_pd,
    cauchy_stress_pd,
    potential_energy_pd,
)

findspark.init()
format = "jdbc"
load_dotenv("./.env")

True

# Set up MTPU and Carolina Materials readers and data

In [2]:
def convert_stress(keys, stress):
    stresses = {k: s for k, s in zip(keys, stress)}
    return [
        [stresses["xx"], stresses["xy"], stresses["xz"]],
        [stresses["xy"], stresses["yy"], stresses["yz"]],
        [stresses["xz"], stresses["yz"], stresses["zz"]],
    ]


SYMBOL_DICT = {"0": "Si", "1": "O"}


def reader(filepath):
    with open(filepath, "rt") as f:
        energy = None
        forces = None
        coords = []
        cell = []
        symbols = []
        config_count = 0
        for line in f:
            if line.strip().startswith("Size"):
                size = int(f.readline().strip())
            elif line.strip().lower().startswith("supercell"):
                cell.append([float(x) for x in f.readline().strip().split()])
                cell.append([float(x) for x in f.readline().strip().split()])
                cell.append([float(x) for x in f.readline().strip().split()])
            elif line.strip().startswith("Energy"):
                energy = float(f.readline().strip())
            elif line.strip().startswith("PlusStress"):
                stress_keys = line.strip().split()[-6:]
                stress = [float(x) for x in f.readline().strip().split()]
                stress = convert_stress(stress_keys, stress)
            elif line.strip().startswith("AtomData:"):
                keys = line.strip().split()[1:]
                if "fx" in keys:
                    forces = []
                for i in range(size):
                    li = {
                        key: val for key, val in zip(keys, f.readline().strip().split())
                    }
                    symbols.append(SYMBOL_DICT[li["type"]])
                    if "cartes_x" in keys:
                        coords.append(
                            [
                                float(c)
                                for c in [
                                    li["cartes_x"],
                                    li["cartes_y"],
                                    li["cartes_z"],
                                ]
                            ]
                        )
                    elif "direct_x" in keys:
                        coords.append(
                            [
                                float(c)
                                for c in [
                                    li["direct_x"],
                                    li["direct_y"],
                                    li["direct_z"],
                                ]
                            ]
                        )

                    if "fx" in keys:
                        forces.append(
                            [float(f) for f in [li["fx"], li["fy"], li["fz"]]]
                        )

            elif line.startswith("END_CFG"):
                if "cartes_x" in keys:
                    config = AtomicConfiguration(
                        positions=coords, symbols=symbols, cell=cell
                    )
                elif "direct_x" in keys:
                    config = AtomicConfiguration(
                        scaled_positions=coords, symbols=symbols, cell=cell
                    )
                config.info["energy"] = energy
                if forces:
                    config.info["forces"] = forces
                config.info["stress"] = stress

                if "Si" in symbols and "O" in symbols:
                    config.info["input"] = {
                        "kpoint-scheme": "Monkhorst-Pack",
                        "kpoints": "11x11x11",
                        "kinetic-energy-cutoff": {
                            "val": 1224,
                            "units": "eV",
                        },
                    }
                    config.info["_name"] = f"{filepath.stem}_SiO2_{config_count}"
                elif "Si" in symbols:
                    config.info["input"] = {
                        "kpoint-scheme": "Monkhorst-Pack",
                        "kpoints": "8x8x8",
                        "kinetic-energy-cutoff": {
                            "val": 884,
                            "units": "eV",
                        },
                    }
                    config.info["_name"] = f"{filepath.stem}_Si_{config_count}"
                elif "O" in symbols:
                    config.info["input"] = {
                        "kpoint-scheme": "Monkhorst-Pack",
                        "kpoints": "gamma-point",
                        "kinetic-energy-cutoff": {
                            "val": 1224,
                            "units": "eV",
                        },
                    }
                    config.info["_name"] = f"{filepath.stem}_O_{config_count}"
                config_count += 1
                yield config
                forces = None
                stress = []
                coords = []
                cell = []
                symbols = []
                energy = None

In [3]:
# mtpu_configs = reader(Path("data/mtpu_2023/Unified_training_set.cfg"))
# data = [x for x in mtpu_configs]
# data[0].configuration_summary()

In [4]:
# data[0]

In [5]:
SOFTWARE = "VASP"
METHODS = "DFT-PBE"
CM_PI_METADATA = {
    "software": {"value": SOFTWARE},
    "method": {"value": METHODS},
    "input": {"value": {"IBRION": 6, "NFREE": 4}},
}

CM_PROPERTY_MAP = {
    "formation-energy": [
        {
            "energy": {"field": "energy", "units": "eV"},
            "per-atom": {"value": False, "units": None},
        }
    ],
    "_metadata": CM_PI_METADATA,
}
CO_MD = {
    key: {"field": key}
    for key in [
        "_symmetry_space_group_name_H-M",
        "_symmetry_Int_Tables_number",
        "_chemical_formula_structural",
        "_chemical_formula_sum",
        "_cell_volume",
        "_cell_formula_units_Z",
        "symmetry_dict",
        "formula_pretty",
    ]
}


def load_row(txn, row):
    try:
        data = pickle.loads(txn.get(f"{row}".encode("ascii")))
        return data
    except TypeError:
        return False


def config_from_row(row: dict, row_num: int):
    coords = row.pop("cart_coords")
    a_num = row.pop("atomic_numbers")
    cell = [
        row.pop(x)
        for x in [
            "_cell_length_a",
            "_cell_length_b",
            "_cell_length_c",
            "_cell_angle_alpha",
            "_cell_angle_beta",
            "_cell_angle_gamma",
        ]
    ]
    config = AtomicConfiguration(scaled_positions=coords, numbers=a_num, cell=cell)
    symmetry_dict = {str(key): val for key, val in row.pop("symmetry_dict").items()}
    for key in symmetry_dict:
        key = str(key)
    config.info = row
    config.info["symmetry_dict"] = symmetry_dict
    config.info["_name"] = f"carolina_materials_{row_num}"
    return config
    # return AtomicConfiguration.from_ase(config)


def carmat_reader(fp: Path):
    parent = fp.parent
    env = lmdb.open(str(parent))
    txn = env.begin()
    row_num = 0
    rows = []
    while row_num <= 10000:
        row = load_row(txn, row_num)
        if row is False:
            env.close()
            break
        rows.append(row)
        yield config_from_row(row, row_num)
        row_num += 1
    env.close()
    return False
    # return rows

In [6]:
reader = carmat_reader(Path("data/carolina_matdb/base/all/data.mdb"))

In [7]:
carmat_configs = list(carmat_reader(Path("data/carolina_matdb/base/all/data.mdb")))
carmat_config_gen = carmat_reader(Path("data/carolina_matdb/base/all/data.mdb"))
PI_METADATA = {
    "software": {"value": "Quantum ESPRESSO"},
    "method": {"value": "DFT-PBE"},
    "input": {"field": "input"},
}

PROPERTY_MAP = {
    "potential-energy": [
        {
            "energy": {"field": "energy", "units": "eV"},
            "per-atom": {"value": False, "units": None},
            # "_metadata": PI_METADATA,
        }
    ],
    "atomic-forces": [
        {
            "forces": {"field": "forces", "units": "eV/angstrom"},
            # "_metadata": PI_METADATA,
        },
    ],
    "cauchy-stress": [
        {
            "stress": {"field": "stress", "units": "GPa"},
            "volume-normalized": {"value": True, "units": None},
        }
    ],
    "_metadata": PI_METADATA,
}

In [8]:
print(carmat_configs[0])

AtomicConfiguration(name=carolina_materials_0, symbols='BrCaH6Rh2', pbc=False, cell=[[5.39874426, 0.0, 0.0], [2.6993721300000004, 4.67544967769542, 0.0], [2.6993721300000004, 1.5584832258984738, 4.4080562295931855]])


In [9]:
from colabfit.tools.configuration import AtomicConfigurationOld

In [10]:
# carmat_aconfigs = [
#     AtomicConfiguration(co_md_map=CO_MD, names="test_name").from_ase(c)
#     for c in carmat_configs
# ]
# carmat_oconfigs = [
#     AtomicConfigurationOld(names=f"{i}").from_ase(c)
#     for i, c in enumerate(carmat_configs)
# ]
# for c in carmat_aconfigs:
#     c.set_metadata(CO_MD)
# len(carmat_aconfigs)

# Connect to DB and run loader

In [26]:
JARFILE = os.environ.get("CLASSPATH")
spark = (
    SparkSession.builder.appName("PostgreSQL Connection with PySpark")
    .config("spark.jars", JARFILE)
    .getOrCreate()
)
url = "jdbc:postgresql://localhost:5432/colabfit"
user = os.environ.get("PGS_USER")
password = os.environ.get("PGS_PASS")
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver",
}
loader = PGDataLoader(appname="colabfit", env="./.env")

24/04/18 13:34:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/04/18 13:34:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [12]:
carmat_config_gen = carmat_reader(Path("data/carolina_matdb/base/all/data.mdb"))

dm = DataManager(
    nprocs=4,
    configs=carmat_config_gen,
    prop_defs=[formation_energy_pd],
    prop_map=CM_PROPERTY_MAP,
)

In [13]:
# carmat_configs = list(carmat_reader(Path("data/carolina_matdb/base/all/data.mdb")))
# dm = DataManager(
#     nprocs=4,
#     configs=carmat_configs,
#     prop_defs=[formation_energy_pd],
#     prop_map=CM_PROPERTY_MAP,
# )

In [23]:
dm.load_data_to_pg_in_batches(loader)

number of chunks 4
1000


24/04/18 11:18:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


number of chunks 4
1000
number of chunks 3
1000


In [63]:
loader.url

'jdbc:postgresql://localhost:5432/colabfit'

In [83]:
df = loader.spark.read.format("jdbc").options(
    url=loader.url,
    table="public.configurations",
    properties=loader.properties,
)

In [131]:
df = (
    loader.spark.read.jdbc(
        url=loader.url, table="public.configurations", properties=loader.properties
    )
    # .select("elements", "elements_ratios", "id", "dataset_ids")
    # .where("dataset_ids is null")
)

In [144]:
df.agg({"nsites": "sum"}).first()[0]

443925

In [145]:
import pyspark.sql.functions as sf

In [153]:
def unstringify(x):
    try:
        return json.loads(x)
    except json.JSONDecodeError:
        return x

In [159]:
df.withColumn(
    "elements",
    sf._from_numpy_type(
        "elements",
    ),
).show()

PySparkTypeError: [NOT_COLUMN] Argument `col` should be a Column, got NoneType.

In [152]:
df.agg(sf.collect_set("elements")).first()[0]

["['Ge', 'Hf', 'Pb', 'Rb']",
 "['H', 'In', 'Sc', 'Si']",
 "['F', 'Pd', 'Rh', 'Te']",
 "['Al', 'Cu', 'Pb', 'Tc']",
 "['Be', 'C', 'Fe', 'N']",
 "['Cr', 'Sn', 'Te', 'V']",
 "['Ag', 'Cd', 'O', 'Zn']",
 "['Be', 'Fe', 'Pb', 'Pd']",
 "['Cr', 'I', 'Rh', 'Zr']",
 "['Ag', 'C', 'Mg', 'Tc']",
 "['Os', 'Ta', 'V', 'Zr']",
 "['Au', 'Hg', 'Ru', 'Ta']",
 "['As', 'Bi', 'Rh', 'Zn']",
 "['Be', 'H', 'Pb', 'Sb']",
 "['Al', 'Co', 'Pd', 'W']",
 "['Ga', 'I', 'Mn', 'Ru']",
 "['F', 'O', 'Rh', 'S']",
 "['As', 'Os', 'Ta', 'Zn']",
 "['Hg', 'Mo', 'Ti', 'Y']",
 "['B', 'Li', 'Rh', 'Ti']",
 "['Br', 'Ca', 'P', 'Rh']",
 "['Ni', 'Rb', 'S', 'Si']",
 "['Al', 'Re', 'Te', 'W']",
 "['Au', 'Cd', 'Os', 'Pb']",
 "['Br', 'Cl', 'Ta', 'V']",
 "['I', 'Li', 'Pb', 'Tc']",
 "['Br', 'Cr', 'Sn', 'Sr']",
 "['Bi', 'Cr', 'Ir', 'Os']",
 "['Au', 'Ba', 'Sb', 'Sc']",
 "['Cr', 'Mg', 'Ti', 'Zn']",
 "['Al', 'Ge', 'In', 'W']",
 "['Au', 'Be', 'Fe', 'Mo']",
 "['Ge', 'In', 'Os', 'S']",
 "['Ga', 'Ni', 'Sb', 'W']",
 "['Be', 'Cu', 'Os', 'Pt']",
 "['Al', '

In [132]:
df.show()

+--------------------+-------------------+-------------------+-----------+--------------------+---------------------+------------------------+--------------------------+--------------------+--------------------+--------------------+------+---------+--------------------+--------------------+---------------+-------------------+--------------------+-----+
|                  id|               hash|      last_modified|dataset_ids|            metadata|chemical_formula_hill|chemical_formula_reduced|chemical_formula_anonymous|            elements|     elements_ratios|      atomic_numbers|nsites|nelements|nperiodic_dimensions|                cell|dimension_types|                pbc|           positions|names|
+--------------------+-------------------+-------------------+-----------+--------------------+---------------------+------------------------+--------------------------+--------------------+--------------------+--------------------+------+---------+--------------------+--------------------

In [93]:
df.filter(df.metadata.isNull()).show()

+--------------------+-------------------+-------------------+-----------+--------+---------------------+------------------------+--------------------------+--------------------+--------------------+--------------------+------+---------+--------------------+--------------------+---------------+-------------------+--------------------+-----+
|                  id|               hash|      last_modified|dataset_ids|metadata|chemical_formula_hill|chemical_formula_reduced|chemical_formula_anonymous|            elements|     elements_ratios|      atomic_numbers|nsites|nelements|nperiodic_dimensions|                cell|dimension_types|                pbc|           positions|names|
+--------------------+-------------------+-------------------+-----------+--------+---------------------+------------------------+--------------------------+--------------------+--------------------+--------------------+------+---------+--------------------+--------------------+---------------+-------------------

In [48]:
df = loader.spark.read.jdbc(
    loader.url,
    table="public.configurations",
    properties=loader.properties,
).options(query="SELECT * FROM public.configurations")

AttributeError: 'DataFrame' object has no attribute 'options'

In [42]:
loader.spark.read.format("jdbc").options(
    url=loader.url,
    query="SELECT * FROM public.configurations",
).load()

Py4JJavaError: An error occurred while calling o207.load.
: java.lang.NoSuchMethodException: org.apache.spark.sql.execution.datasources.jdbc.DriverWrapper.<init>()
	at java.base/java.lang.Class.getConstructor0(Class.java:3349)
	at java.base/java.lang.Class.getConstructor(Class.java:2151)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:54)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:157)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1$adapted(JdbcDialects.scala:156)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:63)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:58)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.getSchema(JDBCRelation.scala:241)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvide

In [38]:
pyspark.sql.SQLContext(loader.spark).sql("SELECT * FROM configurations").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `configurations` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [configurations], [], false


# Hashing

# outer

In [ ]:
"""
Can we make the configuration and the property instance/data object at the same time?
In this way, we would only have to pass through the data one time.

Workflow:
create database access object
create data reader as function? of the database access object
reader returns ase.Atoms-style objects (AtomicConfiguration)
DOs and PIs are now one object
These DOs point to a configuration
The configuration may already exist in the database, so we keep track of the hash added to the DO


"""

In [ ]:
cos = json.load(Path("sample_db/co_ds1.json").open("r"))

In [ ]:
with open(Path("sample_db/co_ds1.json"), "r") as f:
    co_json = spark.sparkContext.parallelize(json.load(f))

In [ ]:
co = co_json.map(_parse_config).map(stringify_lists)
co_df = spark.createDataFrame(co, config_schema)

In [ ]:
def parse_configs(co_path, spark):
    with open(co_path, "r") as f:
        co_json = spark.sparkContext.parallelize(json.load(f))
    co = co_json.map(_parse_config).map(stringify_lists)
    co_df = spark.createDataFrame(co, config_schema)
    return co_df

In [ ]:
parse_configs("sample_db/co_ds1.json", spark).show()

In [ ]:
table_name = "co"

mode = "append"
url = "jdbc:postgresql://localhost:5432/colabfit"
properties = {"user": user, "password": password, "driver": "org.postgresql.Driver"}
co_df.write.jdbc(url=url, table=table_name, mode=mode, properties=properties)

In [ ]:
# co_df.write.